In [1]:
import pandas as pd
from glob import glob
import os
from tqdm.notebook import tqdm


In [2]:
importance_col = "RF importance (permute)"
top_n = 10


In [3]:
results_paths = glob("results/results_*")
results_paths


['results/results_ICPSR_1992_EPEST_2003',
 'results/results_ICPSR_1992_EPEST_2002',
 'results/results_ICPSR_1992_EPEST_1997',
 'results/results_ICPSR_1992_EPEST_2001']

In [4]:
data_folder = "/Users/dslim/pesticide_rf_xgboost/data"
data_path = os.path.join(data_folder, "1992_Tab_v2/DS0042/35206-0042-Data.tsv")

feature_df = pd.read_csv(data_path, sep="\t")
feature_df = feature_df.set_index("fips")
cols_to_drop = [col for col in feature_df.columns if "flag" in col.lower()]
feature_df = feature_df.drop(columns=cols_to_drop)
feature_df.head()


<ipython-input-4-ea461949017c>:4: DtypeWarning: Columns (71,73,507,509,1461,1463) have mixed types. Specify dtype option on import or set low_memory=False.
  feature_df = pd.read_csv(data_path, sep="\t")


,state,county,name,level,statefip,counfip,item010001,item010002,item010003,item010004,...,item390003,item390004,item390005,item390006,item390007,item390008,item390009,item390010,item390011,item390012
fips,,,,,,,,,,,,,,,,,,,,,
0,100,0,UNITED STATES,3,0,0,1925300,945531520,491,357056,...,5133,4056,3186,3670,3701,10498,9232,12927,2000,11831
1000,41,0,ALABAMA,2,1,0,37905,8450823,223,220265,...,56,16,3,3,3,0,15,11,10,60
1001,41,10,AUTAUGA,1,1,1,322,107259,333,226198,...,0,0,0,0,0,0,0,0,0,0
1003,41,30,BALDWIN,1,1,3,941,167832,178,240396,...,0,0,0,0,0,0,0,0,0,0
1005,41,50,BARBOUR,1,1,5,421,177189,421,327882,...,0,0,0,0,0,0,0,0,0,0


In [5]:
index_cols = feature_df.columns[:6].tolist()
index_cols


['state', 'county', 'name', 'level', 'statefip', 'counfip']

In [6]:
def get_top_n_features(output_folder, pesticide_name, top_feat_codes):
    top_n_feature_df = feature_df[index_cols + top_feat_codes]

    csv_name = (
        os.path.join(output_folder, pesticide_name) +
        f"_top_{top_n}_features.csv"
    )
    top_n_feature_df = top_n_feature_df[~(top_n_feature_df.county == 0)]
    top_n_feature_df = top_n_feature_df.drop(
        ["state", "county", "level"], axis=1)
    top_n_feature_df.to_csv(csv_name)


def filter_df(feat_importance_df, corr_df, threshold=0.8, top_n=10):
    top_filtered_feature_n = 0
    total_feat_n = feat_importance_df.shape[0]

    feat_importance_df_filtered = feat_importance_df.copy()
    # display(feat_importance_df)
    # display(corr_df)
    while top_filtered_feature_n < top_n:
        feat_code = feat_importance_df_filtered.index[top_filtered_feature_n]
        print(
            f"Filtering {feat_code} ({feat_importance_df_filtered.loc[feat_code, 'feature_name']})"
        )
        # display(corr_df[feat_code])
        feat_to_keep = corr_df[feat_code].loc[
            feat_importance_df_filtered.index.tolist()
        ]

        feat_to_keep = feat_to_keep[feat_to_keep < threshold].index.tolist() + [
            feat_code
        ]
        feat_to_keep_n = len(feat_to_keep)
        print(
            f"Keeping {feat_to_keep_n}/{total_feat_n} ({feat_to_keep_n/total_feat_n*100:.1f}%) features"
        )

        feat_importance_df_filtered = feat_importance_df_filtered.loc[
            feat_to_keep
        ].sort_values(importance_col, ascending=False)
        print(feat_importance_df_filtered.shape)
        # display(feat_importance_df_filtered.head(10))
        top_filtered_feature_n += 1
    return feat_importance_df_filtered


def get_top_n_feature_info(
    output_folder,
    result_path,
    pesticide_name,
    top_n=10,
    corr_threshold=0.9,
    filter_by_corr=True,
    corr_df=None,
):
    feat_importance_df = pd.read_csv(result_path, index_col=0)
    feat_importance_df[importance_col] = pd.to_numeric(
        feat_importance_df[importance_col]
    )

    feat_importance_df.sort_values(
        importance_col, ascending=False, inplace=True)
    # display(feat_importance_df)
    if filter_by_corr:
        feat_importance_df["rank"] = range(1, feat_importance_df.shape[0] + 1)
        feat_importance_df = filter_df(
            feat_importance_df, corr_df, threshold=corr_threshold, top_n=top_n
        )
    feat_importance_df = feat_importance_df.iloc[:top_n]
    top_feats = feat_importance_df["feature_name"]

    top_feats_path = os.path.join(
        output_folder, f"{pesticide_name}_top_{top_n}_feature_info.csv"
    )
    feat_importance_df.to_csv(top_feats_path, index=False)
    top_feat_codes = top_feats.index.tolist()

    return top_feat_codes


In [7]:
corr_threshold = 0.9
filter_by_corr = True


In [8]:
def get_corr(df, method="spearman"):
    return df[[col for col in df.columns if "item" in col]].corr(method=method).abs()


corr_df = None
if filter_by_corr:
    corr_df = get_corr(feature_df, method="spearman")

root_output_folder = f"top_{top_n}_features_filtered"
os.makedirs(root_output_folder, exist_ok=True)


In [9]:
for result_folder in (pbar_outer := tqdm(results_paths)):
    folder_name = os.path.basename(result_folder)
    feature_set_name = folder_name.split("_", 1)[1]
    output_folder = os.path.join(root_output_folder, feature_set_name)
    os.makedirs(output_folder, exist_ok=True)

    pbar_outer.set_description(f"Processing feature set '{feature_set_name}'")
    pat = os.path.join(result_folder, "*_merged.csv")
    result_paths = glob(pat)

    for result_path in (pbar_inner := tqdm(result_paths)):
        result_file_name = os.path.basename(result_path)
        pesticide_name = result_file_name.split("_")[0]

        pbar_inner.set_description(f"Processing {result_file_name}")

        top_feat_codes = get_top_n_feature_info(
            output_folder,
            result_path,
            pesticide_name,
            top_n=top_n,
            filter_by_corr=filter_by_corr,
            corr_df=corr_df,
            corr_threshold=corr_threshold,
        )

        get_top_n_features(output_folder, pesticide_name, top_feat_codes)


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/321 [00:00<?, ?it/s]

Filtering item020078 (SIC-Sugarcane & sugar beets; Irish potatoes; field crops, exceptcash grains, n.e.c. (0133, 0134, 0139), 1992)
Keeping 2748/2781 (98.8%) features
(2748, 6)
Filtering item120027 (Oats, ($1,000), 1992)
Keeping 2748/2781 (98.8%) features
(2748, 6)
Filtering item130112 (Soybeans for beans-Irrigated (acres), 1992)
Keeping 2747/2781 (98.8%) features
(2747, 6)
Filtering item310189 (Pears, Harvested (pounds), 1992)
Keeping 2747/2781 (98.8%) features
(2747, 6)
Filtering item260005 (Corn for grain or seed, Irrigated (acres), 1992)
Keeping 2747/2781 (98.8%) features
(2747, 6)
Filtering item010082 (Cotton (acres), 1992)
Keeping 2730/2781 (98.2%) features
(2730, 6)
Filtering item090040 (Manufactured in the last five years--Cotton pickers & strippers,(numbers), 1992)
Keeping 2730/2781 (98.2%) features
(2730, 6)
Filtering item310343 (Pecans (in shell), Total (farms), 1992)
Keeping 2725/2781 (98.0%) features
(2725, 6)
Filtering item020003 (Total sales, average per farm, (dollars),

  0%|          | 0/320 [00:00<?, ?it/s]

Filtering item020059 (Other crops, ($1,000), 1992)
Keeping 2748/2781 (98.8%) features
(2748, 6)
Filtering item270078 (Peanuts for nuts, Harvested (pounds), 1992)
Keeping 2746/2781 (98.7%) features
(2746, 6)
Filtering item010107 (Peanuts for nuts (pounds), 1992)
Keeping 2745/2781 (98.7%) features
(2745, 6)
Filtering item120043 (Other crops, ($1,000), 1992)
Keeping 2745/2781 (98.7%) features
(2745, 6)
Filtering item260057 (Canola, Harvested (acres), 1992)
Keeping 2743/2781 (98.6%) features
(2743, 6)
Filtering item290078 (Green cowpeas & green southern peas, Harvested (acres), 1992)
Keeping 2742/2781 (98.6%) features
(2742, 6)
Filtering item020043 (Oats, ($1,000), 1992)
Keeping 2742/2781 (98.6%) features
(2742, 6)
Filtering item020058 (Other crops, (farms), 1992)
Keeping 2741/2781 (98.6%) features
(2741, 6)
Filtering item310344 (Pecans (in shell), Total (acres), 1992)
Keeping 2738/2781 (98.5%) features
(2738, 6)
Filtering item040033 (CCC Loan-Corn, ($1,000), 1992)
Keeping 2738/2781 (98.5%

  0%|          | 0/295 [00:00<?, ?it/s]

Filtering item270078 (Peanuts for nuts, Harvested (pounds), 1992)
Keeping 2746/2781 (98.7%) features
(2746, 6)
Filtering item030045 (Property taxes paid, ($1,000), 1992)
Keeping 2745/2781 (98.7%) features
(2745, 6)
Filtering item020059 (Other crops, ($1,000), 1992)
Keeping 2745/2781 (98.7%) features
(2745, 6)
Filtering item120137 (Place of residence-Not on farm operated, 1992)
Keeping 2742/2781 (98.6%) features
(2742, 6)
Filtering item120043 (Other crops, ($1,000), 1992)
Keeping 2742/2781 (98.6%) features
(2742, 6)
Filtering item020052 (Vegetables, sweet corn, & melons (farms), 1992)
Keeping 2736/2781 (98.4%) features
(2736, 6)
Filtering item300002 (Land in orchards, Total (acres), 1992)
Keeping 2735/2781 (98.3%) features
(2735, 6)
Filtering item310351 (Pecans (in shell), Harvested (pounds), 1992)
Keeping 2735/2781 (98.3%) features
(2735, 6)
Filtering item310349 (Pecans (in shell), Trees or vines of bearing age (number), 1992)
Keeping 2734/2781 (98.3%) features
(2734, 6)
Filtering item

  0%|          | 0/314 [00:00<?, ?it/s]

Filtering item020059 (Other crops, ($1,000), 1992)
Keeping 2748/2781 (98.8%) features
(2748, 6)
Filtering item120043 (Other crops, ($1,000), 1992)
Keeping 2748/2781 (98.8%) features
(2748, 6)
Filtering item010107 (Peanuts for nuts (pounds), 1992)
Keeping 2747/2781 (98.8%) features
(2747, 6)
Filtering item270078 (Peanuts for nuts, Harvested (pounds), 1992)
Keeping 2745/2781 (98.7%) features
(2745, 6)
Filtering item150042 (Hogs & pigs sold 200 to 499, (farms), 1992)
Keeping 2741/2781 (98.6%) features
(2741, 6)
Filtering item020043 (Oats, ($1,000), 1992)
Keeping 2741/2781 (98.6%) features
(2741, 6)
Filtering item260083 (Oats for grain, Harvested (bu.), 1992)
Keeping 2737/2781 (98.4%) features
(2737, 6)
Filtering item100012 (Sprays, dusts, granules, fumigants, etc., to control--diseasesin crops & orchards (acres on which used), 1992)
Keeping 2737/2781 (98.4%) features
(2737, 6)
Filtering item250009 (Goats, Total Sales (number), 1992)
Keeping 2737/2781 (98.4%) features
(2737, 6)
Filtering i

In [10]:
feat_importance_df = pd.read_csv(result_path, index_col=0)
feat_importance_df[importance_col] = pd.to_numeric(
    feat_importance_df[importance_col])

feat_importance_df.sort_values(importance_col, ascending=False, inplace=True)
# display(feat_importance_df)
# feat_importance_df = feat_importance_df.iloc[:top_n]


In [11]:
corr_df


,item010001,item010002,item010003,item010004,item010005,item010006,item010007,item010008,item010009,item010010,...,item390003,item390004,item390005,item390006,item390007,item390008,item390009,item390010,item390011,item390012
item010001,1.000000,0.506992,0.067692,0.057044,0.232677,0.130027,0.790803,0.806179,0.888833,0.892464,...,0.320337,0.268546,0.211981,0.126347,0.035885,0.026215,0.073097,0.140846,0.142345,0.297861
item010002,0.506992,1.000000,0.745195,0.491535,0.419074,0.494597,0.282242,0.106796,0.192387,0.477400,...,0.368619,0.367217,0.402080,0.456940,0.477673,0.522425,0.312889,0.396688,0.282739,0.547599
item010003,0.067692,0.745195,1.000000,0.627445,0.594970,0.568417,0.214001,0.406558,0.339459,0.021926,...,0.128479,0.187950,0.261923,0.388783,0.455800,0.537352,0.236244,0.278156,0.150431,0.329562
item010004,0.057044,0.491535,0.627445,1.000000,0.074078,0.634216,0.089109,0.152392,0.319675,0.137044,...,0.040739,0.039858,0.116240,0.208774,0.290147,0.401581,0.158127,0.201955,0.054779,0.134198
item010005,0.232677,0.419074,0.594970,0.074078,1.000000,0.038717,0.488694,0.550609,0.361861,0.106338,...,0.179054,0.236409,0.279876,0.359949,0.398051,0.430171,0.243300,0.261330,0.221921,0.363355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
item390008,0.026215,0.522425,0.537352,0.401581,0.430171,0.153169,0.005191,0.146815,0.233309,0.134242,...,0.484029,0.487524,0.538949,0.651441,0.738502,1.000000,0.614985,0.639508,0.377380,0.648355
item390009,0.073097,0.312889,0.236244,0.158127,0.243300,0.031915,0.087518,0.049626,0.011158,0.008434,...,0.631174,0.616939,0.612035,0.615975,0.613048,0.614985,1.000000,0.712781,0.485458,0.605038
item390010,0.140846,0.396688,0.278156,0.201955,0.261330,0.019690,0.158470,0.081698,0.008714,0.023732,...,0.683461,0.636079,0.621797,0.634767,0.623742,0.639508,0.712781,1.000000,0.481370,0.680386
item390011,0.142345,0.282739,0.150431,0.054779,0.221921,0.012022,0.109884,0.062566,0.053964,0.082748,...,0.480652,0.437279,0.441958,0.423365,0.420079,0.377380,0.485458,0.481370,1.000000,0.488882
